In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import torch 
from argparse import Namespace
from tqdm import tqdm
import pickle 
import glob 
import ast

# Add the path to the directory containing the sybil module
sys.path.append('/workspace/home/tengyuezhang/sybil_cect/code/Sybil/')
from sybil.utils.metrics import concordance_index, get_survival_metrics
from sybil import Sybil, Serie
from sybil import visualize_attentions


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
num_threads = os.cpu_count() // 5

In [4]:
IN_CASES_PATH = '/workspace/home/tengyuezhang/sybil_cect/data/ucla_cect/ucla_cect_baseline_cases.csv'
OUT_FEATURE_PATH = '/workspace/home/tengyuezhang/sybil_cect/results/ucla_cect/ucla_cect_98_raw_features.csv'

In [5]:
# Initialize the Sybil model
model = Sybil("sybil_1")
num_years = 6

/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Running one example

In [6]:
# Load the CSV file
all_cases = pd.read_csv(IN_CASES_PATH)
example = all_cases.iloc[1]
example

pid                                                         10056_1_327YHR52
timepoint                                                         2020-09-04
scan                       1.3.12.2.1107.5.1.4.73938.30050020071015435508...
AccessionNumber                                                   A_0OTOMW6X
SliceThickness                                                           1.0
ConvolutionKernel                                                      Br46f
ImageOrientationPatient                                   [1, 0, 0, 0, 1, 0]
PatientID                                                   10056_1_327YHR52
Directory                  /workspace/radraid/whsu/idx_data_request_79/Im...
Orientation                                                            axial
Name: 1, dtype: object

In [7]:
dicom_dir = example['Directory']
event = 0
years_to_event = 1
pid = example['pid']
dicom_list = glob.glob(dicom_dir + '/*')
serie = Serie(dicom_list, label=event, censor_time=years_to_event)

results = model.predict([serie], return_attentions=True, threads=num_threads)

In [7]:
results 

Prediction(scores=[[0.012758719037714684, 0.025743208030524028, 0.040977454787905605, 0.05335478429725567, 0.06677989107741973, 0.10217879263786658]], attentions=[{'image_attention_1': array([[[[-11.470681, -10.07798 ,  -9.133754, ...,  -9.066321,
          -10.46895 , -12.185319],
         [-14.458355, -13.403002, -12.598696, ..., -12.14016 ,
          -14.100868, -15.431726],
         [-17.986788, -17.2131  , -16.881214, ..., -15.980452,
          -17.702059, -18.7915  ],
         ...,
         [-18.078018, -17.800737, -18.165924, ..., -17.937977,
          -18.665146, -19.214975],
         [-20.353813, -20.045208, -20.29802 , ..., -19.706808,
          -20.791616, -21.331861],
         [-16.724674, -16.201244, -16.207844, ..., -16.025623,
          -16.947964, -17.385353]]]], dtype=float32), 'volume_attention_1': array([[[-11.6407585, -11.967636 , -12.494825 , -12.254294 ,
         -11.702682 , -12.524836 , -11.818853 ,  -7.42701  ,
          -7.7739034,  -6.7461376,  -5.168269 ,  -

In [8]:
results.attentions[0]['features'][0, 0, :].tolist()

[-0.996924638748169,
 -0.24724116921424866,
 -0.1451576054096222,
 -0.4557493031024933,
 -0.40156227350234985,
 -1.0191634893417358,
 -0.34876951575279236,
 -0.5693208575248718,
 -0.17329853773117065,
 5.099407196044922,
 4.189969062805176,
 -0.12792639434337616,
 -0.2336098849773407,
 -0.08022262901067734,
 -0.4083849787712097,
 -0.392295777797699,
 -0.3131001889705658,
 0.04150423780083656,
 -0.16539950668811798,
 -1.0371801853179932,
 -0.47772982716560364,
 -0.2876025140285492,
 -0.043370623141527176,
 -0.033319566398859024,
 -0.08222568035125732,
 -0.1935325264930725,
 -0.08015225827693939,
 3.1899802684783936,
 0.9302515983581543,
 -0.28981462121009827,
 -2.9689841270446777,
 -0.795571506023407,
 -0.7838329076766968,
 -0.23678551614284515,
 -2.1063365936279297,
 -0.0017286315560340881,
 -0.6068207621574402,
 -2.451246738433838,
 1.7143911123275757,
 2.786296844482422,
 0.02837122417986393,
 -0.22373884916305542,
 -0.4724296033382416,
 -1.0213133096694946,
 -0.8410581350326538,
 -1

## On the entire dataset

In [8]:
# Load the CSV file
all_cases = pd.read_csv(IN_CASES_PATH)
df = all_cases # initialize the output df with the input csv file 
num_features = 512 

In [9]:
for i in range(num_years):
    df[f'pred_risk_year_{i}'] = np.nan
for i in range(num_features):
    df[f'feature_{i}'] = np.nan 

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing cases"):
    dicom_dir = row['Directory']
    event = 0
    years_to_event = 1
    pid = row['pid']
    dicom_list = glob.glob(dicom_dir + '/*')
    serie = Serie(dicom_list, label=event, censor_time=years_to_event)
    
    # get predicted risk scores and features from the last hidden layer (returned along with the attentions)
    results = model.predict([serie], return_attentions=True, threads=num_threads)
        
    # append risk scores 
    for i in range(num_years):
        df.at[index, f'pred_risk_year_{i}'] = results.scores[0][i]
    
    # append features 
    for i in range(num_features):
        # 'feature' for before relu 
        #'hidden' for after relu
        df.at[index, f'feature_{i}'] = results.attentions[0]['features'][0, 0, i] 
        
    # save updated df 
    df.to_csv(OUT_FEATURE_PATH, index=False)

/tmp/ipykernel_15948/2085499521.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'feature_{i}'] = np.nan
/tmp/ipykernel_15948/2085499521.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'feature_{i}'] = np.nan
/tmp/ipykernel_15948/2085499521.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.cop

Processing cases: 100%|███████████████████████████████████████████████████████████████████| 131/131 [21:33<00:00,  9.87s/it]


In [11]:
df

,pid,timepoint,scan,AccessionNumber,SliceThickness,ConvolutionKernel,ImageOrientationPatient,PatientID,Directory,Orientation,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,10056_1_2D6033B8,2021-11-07,1.3.12.2.1107.5.1.4.73938.30050020062815422282...,A_64VM3K16,1.0,Br46f,"[1, 0, 0, 0, 1, 0]",10056_1_2D6033B8,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,0.303830,-0.835284,-1.956222,3.572044,-0.662226,-2.543079,2.381283,-0.087851,-0.813700,-0.651338
1,10056_1_327YHR52,2020-09-04,1.3.12.2.1107.5.1.4.73938.30050020071015435508...,A_0OTOMW6X,1.0,Br46f,"[1, 0, 0, 0, 1, 0]",10056_1_327YHR52,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,-0.064466,-0.801965,-1.804150,2.826035,-0.708084,-1.613675,1.798369,-0.140270,-0.986793,-0.401306
2,10056_1_35287EBF,2017-11-09,1.3.12.2.1107.5.1.4.55034.30000017070611472115...,A_T7616624,1.0,B45f,"[1, 0, 0, 0, 1, 0]",10056_1_35287EBF,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,-0.005422,-0.969097,-1.347557,2.342428,-0.624411,-1.419659,1.171240,-0.087398,-0.893408,-0.493792
3,10056_1_3OQZM43U,2017-07-21,CT_Chest_at_TLC_Supine_Post_Contrast--1.3.12.2...,A_K9B244N8,1.0,B45f,"[1, 0, 0, 0, 1, 0]",10056_1_3OQZM43U,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,0.076439,-0.693004,-1.801153,3.298895,-0.755902,-2.105121,2.017926,-0.093280,-0.945839,-0.553289
4,10056_1_4234Y2Y8,2021-05-10,1.3.12.2.1107.5.1.4.75487.30000021032914311465...,A_691JCELP,1.0,"['Br49d', '3']","[1, 0, 0, 0, 1, 0]",10056_1_4234Y2Y8,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,0.090609,-0.556021,-1.643305,2.983127,-0.658228,-2.065290,1.791759,-0.087687,-0.856149,-0.592002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,10064_1_XU847S6B,2015-08-14,1.3.12.2.1107.5.1.4.65225.30000015052916523008...,A_78413Q71,1.0,B45f,"[1, 0, 0, 0, 1, 0]",10064_1_XU847S6B,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,0.237863,-0.426184,-1.688868,1.869588,-0.586244,-1.833009,1.036692,-0.059726,-0.831289,-0.562895
127,10064_1_Y58EADQ2,2023-08-21,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_E1L9A1K8,1.0,Br46f,"[1, 0, 0, 0, 1, 0]",10064_1_Y58EADQ2,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,-0.248252,-0.699202,-1.139538,1.988458,-0.607614,-0.747704,1.277068,-0.093184,-0.872658,-0.315385
128,10064_1_Y6357FFF,2022-07-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_WU680PX8,1.0,B45f,"[1, 0, 0, 0, 1, 0]",10064_1_Y6357FFF,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,-0.121620,-0.609215,-1.561879,3.240826,-0.629420,-1.807782,1.703032,-0.110444,-0.887927,-0.439825
129,10064_1_YRU26VP4,2021-10-27,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_2344T37V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",10064_1_YRU26VP4,/workspace/radraid/whsu/idx_data_request_79/Im...,axial,...,-0.045863,-0.632902,-1.354740,2.621143,-0.626972,-0.977539,1.055454,-0.102271,-0.945472,-0.465843


## Append diagnosis column 

In [19]:
SCAN_SELECT_PATH = '/workspace/home/tengyuezhang/sybil_cect/data/ucla_cect/ucla_cect_98_baseline_timepoint.csv'
scan_select_df = pd.read_csv(SCAN_SELECT_PATH)
scan_select_df

,pid,timepoint,scan,AccessionNumber,SliceThickness,ConvolutionKernel,ImageOrientationPatient,Directory,Orientation,lung_cancer
0,10057_1_082F659V,2019-10-23,CT_Full_(thin)_Chest_at_TLC_Supine_Low_Dose--1...,A_Z3539535,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0
1,10057_1_0GVQOI75,2017-03-28,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.6.1.4....,A_MCVXKR3R,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0
2,10057_1_0Q53DU0X,2018-08-06,CT_Partial_(thin)_Chest_at_TLC_Supine--1.3.12....,A_916QU75X,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0
3,10057_1_0Y2FH7L6,2020-12-14,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_NQ0V8X14,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0
4,10057_1_13OW2TJ4,2016-09-20,CT_Full_(thin)_Chest_at_TLC_Supine_Low_Dose--1...,A_NJ4CA918,1.0,B50f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0
...,...,...,...,...,...,...,...,...,...,...
93,10064_1_WZ7V156P,2022-12-30,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_3AZZ8L7V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0
94,10064_1_XSOLE23B,2023-11-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_84Q4077Z,1.0,Br46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0
95,10064_1_Y6357FFF,2022-07-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_WU680PX8,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0
96,10064_1_YRU26VP4,2021-10-27,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_2344T37V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0


In [20]:
merged_df = scan_select_df.merge(
    df,
    on='pid',
    how='left'
)
merged_df

,pid,timepoint_x,scan_x,AccessionNumber_x,SliceThickness_x,ConvolutionKernel_x,ImageOrientationPatient_x,Directory_x,Orientation_x,lung_cancer,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,10057_1_082F659V,2019-10-23,CT_Full_(thin)_Chest_at_TLC_Supine_Low_Dose--1...,A_Z3539535,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0,...,-0.354582,-0.771911,-1.029119,2.273743,-0.626348,-0.579312,0.719628,-0.103131,-0.945605,-0.304786
1,10057_1_0GVQOI75,2017-03-28,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.6.1.4....,A_MCVXKR3R,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0,...,-0.258374,-0.181273,-1.243455,2.457618,-0.685460,-1.089944,0.599725,-0.112920,-0.992032,-0.351367
2,10057_1_0Q53DU0X,2018-08-06,CT_Partial_(thin)_Chest_at_TLC_Supine--1.3.12....,A_916QU75X,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.474760,0.032642,-0.914653,1.435643,-0.631582,-0.252761,0.245356,-0.103082,-0.990130,-0.202050
3,10057_1_0Y2FH7L6,2020-12-14,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_NQ0V8X14,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.078262,-0.648639,-1.270512,2.920637,-0.698843,-1.228455,1.588717,-0.094295,-0.997826,-0.464302
4,10057_1_13OW2TJ4,2016-09-20,CT_Full_(thin)_Chest_at_TLC_Supine_Low_Dose--1...,A_NJ4CA918,1.0,B50f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.331675,-0.541107,-1.382156,2.149262,-0.621658,-0.844771,1.166961,-0.104866,-0.984679,-0.381093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,10064_1_WZ7V156P,2022-12-30,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_3AZZ8L7V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0,...,0.026597,-0.454528,-1.408085,2.676486,-0.543859,-1.895220,2.027103,-0.075904,-0.837401,-0.523678
94,10064_1_XSOLE23B,2023-11-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_84Q4077Z,1.0,Br46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.334973,-0.236297,-1.272780,2.428913,-0.639197,-0.773067,1.270334,-0.114218,-0.914931,-0.298124
95,10064_1_Y6357FFF,2022-07-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_WU680PX8,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.121620,-0.609215,-1.561879,3.240826,-0.629420,-1.807782,1.703032,-0.110444,-0.887927,-0.439825
96,10064_1_YRU26VP4,2021-10-27,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_2344T37V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.045863,-0.632902,-1.354740,2.621143,-0.626972,-0.977539,1.055454,-0.102271,-0.945472,-0.465843


In [21]:
merged_df

,pid,timepoint_x,scan_x,AccessionNumber_x,SliceThickness_x,ConvolutionKernel_x,ImageOrientationPatient_x,Directory_x,Orientation_x,lung_cancer,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,10057_1_082F659V,2019-10-23,CT_Full_(thin)_Chest_at_TLC_Supine_Low_Dose--1...,A_Z3539535,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0,...,-0.354582,-0.771911,-1.029119,2.273743,-0.626348,-0.579312,0.719628,-0.103131,-0.945605,-0.304786
1,10057_1_0GVQOI75,2017-03-28,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.6.1.4....,A_MCVXKR3R,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0,...,-0.258374,-0.181273,-1.243455,2.457618,-0.685460,-1.089944,0.599725,-0.112920,-0.992032,-0.351367
2,10057_1_0Q53DU0X,2018-08-06,CT_Partial_(thin)_Chest_at_TLC_Supine--1.3.12....,A_916QU75X,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.474760,0.032642,-0.914653,1.435643,-0.631582,-0.252761,0.245356,-0.103082,-0.990130,-0.202050
3,10057_1_0Y2FH7L6,2020-12-14,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_NQ0V8X14,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.078262,-0.648639,-1.270512,2.920637,-0.698843,-1.228455,1.588717,-0.094295,-0.997826,-0.464302
4,10057_1_13OW2TJ4,2016-09-20,CT_Full_(thin)_Chest_at_TLC_Supine_Low_Dose--1...,A_NJ4CA918,1.0,B50f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.331675,-0.541107,-1.382156,2.149262,-0.621658,-0.844771,1.166961,-0.104866,-0.984679,-0.381093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,10064_1_WZ7V156P,2022-12-30,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_3AZZ8L7V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,1.0,...,0.026597,-0.454528,-1.408085,2.676486,-0.543859,-1.895220,2.027103,-0.075904,-0.837401,-0.523678
94,10064_1_XSOLE23B,2023-11-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_84Q4077Z,1.0,Br46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.334973,-0.236297,-1.272780,2.428913,-0.639197,-0.773067,1.270334,-0.114218,-0.914931,-0.298124
95,10064_1_Y6357FFF,2022-07-10,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_WU680PX8,1.0,B45f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.121620,-0.609215,-1.561879,3.240826,-0.629420,-1.807782,1.703032,-0.110444,-0.887927,-0.439825
96,10064_1_YRU26VP4,2021-10-27,CT_Full_(thin)_Chest_at_TLC_Supine--1.3.12.2.1...,A_2344T37V,1.0,B46f,"[1, 0, 0, 0, 1, 0]",/workspace/radraid/whsu/idx_data_request_79/Im...,axial,0.0,...,-0.045863,-0.632902,-1.354740,2.621143,-0.626972,-0.977539,1.055454,-0.102271,-0.945472,-0.465843


In [22]:
merged_df.to_csv(OUT_FEATURE_PATH, index=False)